In [ ]:
# https://leetcode.com/problems/restaurant-growth/

"""
WITH amount_cte AS (
    SELECT
        visited_on,
        SUM(amount) AS daily_total
    FROM Customer
    GROUP BY visited_on
),
averages_cte AS (
    SELECT
        visited_on,
        SUM(daily_total) OVER (ORDER BY visited_on ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS amount,
        ROUND(AVG(daily_total) OVER (ORDER BY visited_on ROWS BETWEEN 6 PRECEDING AND CURRENT ROW), 2) AS average_amount
    FROM amount_cte
)

SELECT *
FROM averages_cte
WHERE visited_on >= (SELECT MIN(visited_on) FROM Customer) + 6
ORDER BY visited_on ASC
"""

# 

"""
WITH last_6_days AS (
    SELECT DISTINCT visited_on
    FROM Customer
    ORDER BY visited_on ASC
    OFFSET 6
)

SELECT c1.visited_on,
        SUM(c2.amount) AS amount,
        ROUND(SUM(c2.amount) / 7., 2)  AS average_amount
FROM last_6_days AS c1
INNER JOIN Customer AS c2
ON c2.visited_on BETWEEN c1.visited_on - 6 AND c1.visited_on
GROUP BY c1.visited_on;
"""

In [ ]:
# https://leetcode.com/problems/market-analysis-i/
"""
SELECT
    u.user_id AS buyer_id,
    u.join_date,
    COUNT(o.order_id) AS orders_in_2019
FROM Users u
    LEFT JOIN Orders o
        ON o.buyer_id = u.user_id
        AND EXTRACT(YEAR FROM o.order_date) = 2019
GROUP BY u.user_id, u.join_date
"""

# 

"""
/* Write your T-SQL query statement below */
with cte as (
select buyer_id,count(*) as orders_in_2019
from orders 
where year(order_date)=2019
group by buyer_id )



select user_id as buyer_id ,join_date,coalesce(orders_in_2019 ,0) as orders_in_2019
from cte c right join users u 
on u.user_id =c.buyer_id 
"""

In [ ]:
# https://leetcode.com/problems/investments-in-2016/

"""
WITH cte AS (
    SELECT 
        *,
        COUNT(*) OVER(PARTITION BY lat, lon) AS location_counter,
        COUNT(*) OVER(PARTITION BY tiv_2015) AS tiv_2015_counter
    FROM Insurance
)

SELECT ROUND(SUM(tiv_2016)::numeric, 2) AS tiv_2016
FROM cte
WHERE
    location_counter = 1
    AND
    tiv_2015_counter > 1;
    """

In [ ]:
# https://leetcode.com/problems/managers-with-at-least-5-direct-reports/

"""
WITH report_count_cte AS (
    SELECT managerId, COUNT(*) AS report_count
    FROM Employee
    GROUP BY managerId
)

SELECT e.name
FROM Employee AS e
    JOIN report_count_cte AS cte
        ON e.id = cte.managerId
WHERE cte.report_count >= 5
"""

In [ ]:
# https://leetcode.com/problems/find-users-with-high-token-usage/

"""
WITH agg_cte AS (   
    SELECT
        user_id,
        COUNT(*) AS prompt_count,
        ROUND(AVG(tokens), 2) AS avg_tokens,
        MAX(tokens) AS max_tokens
    FROM prompts
    GROUP BY user_id
)

SELECT user_id, prompt_count, avg_tokens
FROM agg_cte
WHERE
    prompt_count >= 3
    AND
    max_tokens > avg_tokens
ORDER BY avg_tokens DESC, user_id ASC;
"""

In [ ]:
# https://leetcode.com/problems/find-products-with-valid-serial-numbers/

"""
SELECT 
    p.product_id,
    p.product_name,
    p.description
FROM products p
WHERE p.description ~ '\ySN\d{4}-\d{4}\y'
ORDER BY product_id;
"""

In [ ]:
# https://leetcode.com/problems/find-books-with-polarized-opinions/

"""
WITH aggregate_cte AS (
    SELECT
        book_id,
        (MAX(session_rating) - MIN(session_rating)) AS rating_spread,
        COUNT(*) FILTER (WHERE session_rating <= 2 OR session_rating >= 4) * 1.0
        /
        COUNT(*) AS polarization_score
    FROM reading_sessions
    GROUP BY book_id
    HAVING COUNT(*) >= 5 AND MIN(session_rating) <= 2 AND MAX(session_rating) >= 4
)

SELECT
    b.book_id,
    b.title,
    b.author,
    b.genre,
    b.pages,
    agg.rating_spread,
    ROUND(agg.polarization_score, 2) AS polarization_score
FROM books b
    RIGHT JOIN aggregate_cte agg
        ON b.book_id = agg.book_id
WHERE agg.polarization_score >= 0.6
ORDER BY agg.polarization_score DESC, title DESC;
"""

In [ ]:
# https://leetcode.com/problems/find-books-with-no-available-copies/

"""
WITH current_borrowers_cte AS (
    SELECT lb.book_id, COUNT(*) AS current_borrowers
    FROM library_books lb
        LEFT JOIN borrowing_records br
            ON lb.book_id = br.book_id
    WHERE
        br.borrow_date IS NOT NULL
        AND
        br.return_date IS NULL
    GROUP BY lb.book_id
)

SELECT
    lb.book_id,
    lb.title,
    lb.author,
    lb.genre,
    lb.publication_year,
    cte.current_borrowers
FROM library_books lb
    LEFT JOIN current_borrowers_cte cte
        ON lb.book_id = cte.book_id
WHERE lb.total_copies = cte.current_borrowers
ORDER BY cte.current_borrowers DESC, title ASC;
"""

# UNION ALL method

"""
SELECT book_id, MAX(title) AS title, MAX(author) AS author, MAX(genre) AS genre, MAX(publication_year) AS publication_year, MAX(total_copies) AS current_borrowers FROM
(
    SELECT book_id ,title ,author, genre,publication_year , total_copies,  total_copies AS total_remain FROM library_books
    UNION ALL
    SELECT book_id, '', '', '', 1000, 0, -1 FROM borrowing_records WHERE return_date IS NULL
)sub
GROUP BY book_id
HAVING SUM(total_remain) = 0
ORDER BY current_borrowers DESC, title;
"""


In [ ]:
# https://leetcode.com/problems/find-valid-emails/

"""
SELECT user_id, email
FROM Users
WHERE email ~ '^[a-zA-Z0-9_]+@[a-zA-Z]+\.com$'
ORDER BY user_id ASC;
"""

In [ ]:
# https://leetcode.com/problems/employees-whose-manager-left-the-company/

"""
SELECT
    e.employee_id
FROM Employees e
    LEFT JOIN Employees m
        ON e.manager_id = m.employee_id
WHERE m.employee_id IS NULL AND e.manager_id IS NOT NULL AND e.salary < 30000
ORDER BY e.employee_id;
"""

In [ ]:
# https://leetcode.com/problems/calculate-special-bonus/

"""
SELECT 
    employee_id,
    CASE
        WHEN MOD(employee_id, 2) = 1 AND NOT STARTS_WITH(name, 'M') THEN salary
        ELSE 0
    END AS bonus
FROM Employees
ORDER BY employee_id;
"""

# Use LIKE

"""
SELECT 
    employee_id,
    CASE
        WHEN (MOD(employee_id, 2) = 1) AND (name NOT LIKE 'M%') THEN salary
        ELSE 0
    END AS bonus
FROM Employees
ORDER BY employee_id;
"""

In [ ]:
# https://leetcode.com/problems/seasonal-sales-analysis/

"""
WITH sales_cte AS (
    SELECT 
        CASE
            WHEN EXTRACT(MONTH FROM sale_date) IN (12,1,2) THEN 'Winter'
            WHEN EXTRACT(MONTH FROM sale_date) IN (3,4,5) THEN 'Spring'
            WHEN EXTRACT(MONTH FROM sale_date) IN (6,7,8) THEN 'Summer'
            WHEN EXTRACT(MONTH FROM sale_date) IN (9,10,11) THEN 'Fall'
        END AS season,
        p.category,
        SUM(quantity) AS total_quantity,
        SUM(s.price * s.quantity) AS total_revenue
    FROM sales s
        JOIN products p
            ON s.product_id = p.product_id
    GROUP BY 1, 2
    ORDER BY total_quantity DESC, total_revenue DESC
)

SELECT DISTINCT ON (season)
    season, category, total_quantity, total_revenue
FROM sales_cte
ORDER BY season ASC, total_quantity DESC, total_revenue DESC;
"""

In [ ]:
 # https://leetcode.com/problems/confirmation-rate/

"""
SELECT
    s.user_id,
    ROUND(
        COUNT(
            CASE
                WHEN action = 'confirmed' THEN c.action
            END
        ) * 1.0
        /
        COALESCE(NULLIF(COUNT(c.action), 0), 1)
    , 2) AS confirmation_rate
FROM Signups s
    LEFT JOIN Confirmations c
        ON s.user_id = c.user_id
GROUP BY s.user_id;
"""

# AVG function makes it easier

"""
SELECT
    s.user_id,
    ROUND(
        AVG(
            CASE 
                WHEN c.action = 'confirmed' THEN 1.00
                ELSE 0
            END
        ),
        2
    ) confirmation_rate
FROM
    Signups s
LEFT JOIN
    Confirmations c
    ON
    s.user_id = c.user_id
GROUP BY
    s.user_id
"""

In [ ]:
# https://leetcode.com/problems/find-product-recommendation-pairs

"""
WITH pair_cte AS (
    SELECT
        p1.product_id AS product1_id,
        p2.product_id AS product2_id,
        COUNT(p1.user_id) AS customer_count
    FROM ProductPurchases AS p1
        JOIN ProductPurchases AS p2
            ON p1.product_id < p2.product_id AND p1.user_id = p2.user_id
    GROUP BY product1_id, product2_id
)

SELECT
    product1_id,
    product2_id,
    i1.category AS product1_category,
    i2.category AS product2_category,
    customer_count
FROM pair_cte cte
    JOIN ProductInfo i1
        ON cte.product1_id = i1.product_id
    JOIN ProductInfo i2
        ON cte.product2_id = i2.product_id
    WHERE customer_count >= 3
ORDER BY customer_count DESC, product1_id ASC, product2_id ASC;
"""

In [ ]:
# https://leetcode.com/problems/friend-requests-ii-who-has-the-most-friends/

"""
WITH requester_cte AS (
    SELECT requester_id, COALESCE(COUNT(requester_id), 0) r_count
    FROM RequestAccepted
    GROUP BY requester_id
),
accepter_cte AS (
    SELECT accepter_id, COALESCE(COUNT(accepter_id), 0) as a_count
    FROM RequestAccepted
    GROUP BY accepter_id
)

SELECT COALESCE(r.requester_id, a.accepter_id) as id, COALESCE(r.r_count, 0) + COALESCE(a.a_count, 0) as num
FROM requester_cte r
    FULL JOIN accepter_cte a
        ON r.requester_id = a.accepter_id
ORDER BY num DESC
LIMIT 1;
"""

# The better, more straightforward solution

"""
SELECT id, COUNT(*) as num
FROM (
    SELECT requester_id as id
    FROM RequestAccepted
    UNION ALL
    SELECT accepter_id as id
    FROM RequestAccepted
)
GROUP BY id
ORDER BY num DESC
LIMIT 1;
"""

In [ ]:
# https://leetcode.com/problems/count-salary-categories/

"""
WITH category_cte AS (
    SELECT account_id, income,
        CASE
            WHEN income < 20000 THEN 'Low Salary'
            WHEN 20000 <= income AND income <= 50000 THEN 'Average Salary'
            ELSE 'High Salary'
        END as category,
        1 AS marker
    FROM Accounts
),
category_base_cte AS (
    SELECT UNNEST(ARRAY['Low Salary', 'Average Salary', 'High Salary']) AS category
)

SELECT cb.category, COALESCE(SUM(marker), 0) AS accounts_count
FROM category_base_cte cb
    LEFT JOIN category_cte c
        ON cb.category = c.category
GROUP BY cb.category
"""

# The most simple perhaps

"""
SELECT 'Low Salary' AS category, COUNT(*) AS accounts_count FROM Accounts WHERE income < 20000 
UNION
SELECT 'Average Salary' AS category, COUNT(*) AS accounts_count FROM Accounts WHERE income >= 20000 AND income <= 50000
UNION
SELECT 'High Salary' AS category, COUNT(*) AS accounts_count FROM Accounts WHERE income > 50000 
"""

In [ ]:
# https://leetcode.com/problems/find-churn-risk-customers/

"""
WITH last_event_cte AS (
    SELECT DISTINCT ON (user_id)
        user_id, event_type as last_event, plan_name as current_plan, monthly_amount AS current_monthly_amount
    FROM subscription_events
    ORDER BY user_id, event_id DESC
),

downgrade_cte AS (
    SELECT user_id, TRUE as has_downgraded
    FROM subscription_events
    WHERE event_type = 'downgrade'
),

amount_cte AS (
    SELECT DISTINCT ON (user_id)
        user_id, MAX(monthly_amount) AS max_historical_amount
    FROM subscription_events
    GROUP BY user_id
),

day_count_cte AS (
    SELECT user_id, MAX(event_date::date) - MIN(event_date::date) AS days_as_subscriber
    FROM subscription_events
    WHERE event_type != 'cancel'
    GROUP BY user_id
)

SELECT DISTINCT
    le.user_id,
    current_plan,
    current_monthly_amount,
    max_historical_amount,
    days_as_subscriber
FROM last_event_cte le
    JOIN downgrade_cte d ON le.user_id = d.user_id
    JOIN amount_cte a ON le.user_id = a.user_id
    JOIN day_count_cte dc ON le.user_id = dc.user_id
WHERE le.last_event != 'cancel'
    AND d.has_downgraded = TRUE
    AND 1.0 * le.current_monthly_amount / (a.max_historical_amount) < 0.5
    AND dc.days_as_subscriber > 60
ORDER BY days_as_subscriber DESC, user_id ASC
"""

In [ ]:
# https://leetcode.com/problems/odd-and-even-transactions/

"""
SELECT 
    transaction_date,
    COALESCE(
        SUM(amount) FILTER (WHERE MOD(amount, 2) = 1),
        0
    ) AS odd_sum,
    COALESCE(
        SUM(amount) FILTER (WHERE MOD(amount, 2) = 0),
        0
    ) AS even_sum
FROM transactions
GROUP BY transaction_date
ORDER BY transaction_date;
"""

In [ ]:
# https://leetcode.com/problems/last-person-to-fit-in-the-bus/

"""
WITH cumulative_cte AS (
    SELECT person_id, person_name, weight, turn,
        SUM(weight) OVER (ORDER BY turn ROWS UNBOUNDED PRECEDING) AS cumsum
    FROM Queue
)

SELECT person_name
FROM cumulative_cte
WHERE cumsum <= 1000
ORDER BY turn DESC
LIMIT 1;
"""

In [ ]:
# https://leetcode.com/problems/primary-department-for-each-employee/

"""
WITH employee_cte AS (
    SELECT employee_id, COUNT(*) AS department_count
    FROM Employee
    GROUP BY employee_id
)

SELECT e.employee_id, e.department_id
FROM employee_cte cte
    JOIN Employee e
        ON cte.employee_id = e.employee_id
WHERE e.primary_flag = 'Y' OR cte.department_count = 1;

"""

In [ ]:
# https://leetcode.com/problems/exchange-seats/

"""
SELECT
    CASE
        WHEN id % 2 = 0 THEN id - 1
        WHEN id % 2 = 1 AND id != (SELECT MAX(id) FROM Seat) THEN id + 1
        ELSE id
    END AS id,
    student
FROM Seat
ORDER BY id;
"""

# possible solution:

"""
SELECT 
    id,
    CASE
        WHEN id % 2 = 0 THEN LAG(student) OVER(ORDER BY id)
        ELSE COALESCE(LEAD(student) OVER(ORDER BY id), student)
    END AS student
FROM Seat
"""

In [ ]:
# https://leetcode.com/problems/tree-node/

"""
SELECT
    id,
    CASE
        WHEN p_id IS NULL THEN 'Root'
        WHEN id NOT IN (SELECT p_id FROM Tree WHERE p_id IS NOT NULL) THEN 'Leaf'
        ELSE 'Inner'
    END AS type
FROM Tree;
"""

In [ ]:
# https://leetcode.com/problems/analyze-subscription-conversion/

"""
WITH agg_cte AS (
SELECT
    user_id,
    ROUND(AVG(activity_duration) FILTER (WHERE activity_type = 'free_trial'), 2) AS trial_avg_duration,
    ROUND(AVG(activity_duration) FILTER (WHERE activity_type = 'paid'), 2) AS paid_avg_duration
FROM UserActivity
GROUP BY user_id
)

SELECT *
FROM agg_cte
WHERE
    trial_avg_duration IS NOT NULL
    AND
    paid_avg_duration IS NOT NULL
ORDER BY user_id;
"""

In [ ]:
# https://leetcode.com/problems/the-latest-login-in-2020/

"""
WITH ordered_cte AS (
    SELECT
        user_id,
        time_stamp,
        DENSE_RANK() OVER (PARTITION BY user_id ORDER BY time_stamp DESC) AS rn
    FROM Logins
    WHERE EXTRACT(YEAR FROM time_stamp) = 2020
)

SELECT user_id, time_stamp as last_stamp
FROM ordered_cte
WHERE rn = 1;
"""

In [ ]:
# https://leetcode.com/problems/capital-gainloss/

"""
SELECT
    stock_name,
    SUM(
        CASE
            WHEN operation = 'Buy' THEN -price
            WHEN operation = 'Sell' THEN price
        END
    ) as capital_gain_loss
FROM Stocks
GROUP BY stock_name;
"""

In [ ]:
# https://leetcode.com/problems/dna-pattern-recognition

"""
SELECT sample_id, dna_sequence, species,
    CASE WHEN dna_sequence LIKE 'ATG%' THEN 1 ELSE 0 END AS has_start,
    CASE 
        WHEN 
            dna_sequence LIKE '%TAA' OR
            dna_sequence LIKE '%TAG' OR
            dna_sequence LIKE '%TGA'
        THEN 1
        ELSE 0 
    END AS has_stop,
    CASE WHEN dna_sequence LIKE '%ATAT%' THEN 1 ELSE 0 END AS has_atat,
    CASE WHEN dna_sequence LIKE '%GGG%' THEN 1 ELSE 0 END AS has_ggg
FROM Samples
ORDER BY sample_id ASC;
"""

"""
SELECT sample_id,
       dna_sequence,
       species,
       dna_sequence ~ '^ATG'           AS has_start,
       dna_sequence ~ 'TAA$|TAG$|TGA$' AS has_stop,
       dna_sequence ~ '.*ATAT.*'       AS has_atat,
       dna_sequence ~ '.*GGG.*'        AS has_ggg
  FROM samples
 ORDER BY sample_id;
"""

In [ ]:
# https://leetcode.com/problems/rearrange-products-table

"""
WITH union_cte AS (
SELECT product_id, 'store1' AS store, store1 AS price
FROM Products
UNION ALL
SELECT product_id, 'store2' AS store, store2 AS price
FROM Products
UNION ALL
SELECT product_id, 'store3' AS store, store3 AS price
FROM Products
)

SELECT *
FROM union_cte
WHERE price IS NOT NULL
"""

"""
SELECT 
    product_id, 
    store,
    price
FROM (
  SELECT 
    product_id, 
    UNNEST(ARRAY['store1', 'store2', 'store3']) AS store,
    UNNEST(ARRAY[store1, store2, store3]) AS price
  FROM 
    Products
) AS unpivot
WHERE price IS NOT NULL
"""

In [ ]:
# https://leetcode.com/problems/students-and-examinations/

"""
SELECT st.student_id, st.student_name, sb.subject_name, COALESCE(ec.exam_count, 0) AS attended_exams
FROM
    Students st
    CROSS JOIN Subjects sb 
    LEFT JOIN (SELECT COUNT(*) exam_count, student_id, subject_name FROM Examinations GROUP BY student_id, subject_name) ec
    ON ec.student_id = st.student_id AND ec.subject_name = sb.subject_name
ORDER BY st.student_id, sb.subject_name;
"""

# The count of groups could be done in the main table simply

"""
SELECT
    S.student_id
    ,S.student_name
    ,SU.subject_name
    ,COUNT(E.student_id) attended_exams
FROM Students S
CROSS JOIN Subjects SU
LEFT JOIN Examinations E
    ON S.student_id = E.student_id
    AND SU.subject_name = E.subject_name

GROUP BY S.student_id, S.student_name, SU.subject_name
ORDER BY S.student_id, S.student_name, SU.subject_name
;
"""

In [ ]:
# https://leetcode.com/problems/consecutive-numbers/

"""
SELECT DISTINCT l1.num as ConsecutiveNums
FROM Logs l1
LEFT JOIN Logs l2
ON l1.id = l2.id + 1
LEFT JOIN Logs l3
ON l1.id = l3.id + 2
WHERE l1.num = l2.num and l1.num = l3.num;
"""

# Beautiful solution

"""
SELECT DISTINCT num AS ConsecutiveNums
FROM (
SELECT num, a.id - ROW_NUMBER() OVER (PARTITION BY num ORDER BY id) AS label
FROM Logs a
) cte
GROUP BY label, num
HAVING COUNT(num) >= 3;
"""

In [ ]:
# https://leetcode.com/problems/nth-highest-salary/

"""
CREATE OR REPLACE FUNCTION NthHighestSalary(N INT)
  RETURNS TABLE (Salary INT)
AS $$
BEGIN
  RETURN QUERY
    SELECT
      es1.salary
    FROM      (SELECT DISTINCT e1.salary
        FROM Employee e1) es1
      LEFT JOIN (
        SELECT DISTINCT e2.salary
        FROM Employee e2
      ) es2
        ON es1.salary <= es2.salary
    GROUP BY
      es1.salary
    HAVING
      COUNT(es2.salary) = N;
END;
$$ LANGUAGE plpgsql;
"""

# Joins are not mandatory

"""
CREATE OR REPLACE FUNCTION NthHighestSalary(N INT) RETURNS TABLE (Salary INT) AS $$
BEGIN
IF N < 1 THEN 
    RETURN QUERY(SELECT NULL::INT AS salary);
ELSE
  RETURN QUERY (
    SELECT DISTINCT e.salary
      from Employee e
      ORDER BY salary DESC
      LIMIT 1
      OFFSET N-1
  );
 END IF;
END;
$$ LANGUAGE plpgsql;
"""

In [ ]:
# https://leetcode.com/problems/rank-scores/
"""
SELECT score, DENSE_RANK() over (ORDER BY SCORE DESC) rank FROM Scores;
"""

# A non window function solution
"""
SELECT s1.score, (SELECT COUNT(DISTINCT s2.score) FROM Scores s2 WHERE s2.score >= s1.score) AS rank
FROM Scores s1
ORDER BY s1.score DESC;
"""

# Also possible and better on SQL engines:
"""
SELECT s1.score, COUNT(DISTINCT s2.score) AS rank
FROM Scores s1
LEFT JOIN Scores s2 ON s1.score <= s2.score
GROUP BY s1.score
ORDER BY s1.score DESC;
"""

In [ ]:
# https://leetcode.com/problems/not-boring-movies/

"""SELECT * FROM Cinema
WHERE mod(id,2) = 1 AND description != 'boring'
ORDER BY rating DESC;"""

In [ ]:
# https://leetcode.com/problems/triangle-judgement/submissions/1639090104/

"""
SELECT x, y, z,
(
CASE
  WHEN LEAST(x, y, z) + (x + y + z) - LEAST(x, y, z) - GREATEST(x, y, z) > GREATEST(x, y, z) THEN 'Yes'
  ELSE 'No'
END
) AS triangle
FROM Triangle;
"""

In [ ]:
# https://leetcode.com/problems/sales-person/submissions/1638196688/

"""
SELECT name FROM SalesPerson
WHERE name NOT IN
(SELECT sp.name
FROM SalesPerson sp
LEFT JOIN Orders o
  ON sp.sales_id = o.sales_id
LEFT JOIN Company c
  ON o.com_id = c.com_id
WHERE c.name = 'RED')
"""

In [ ]:
# https://leetcode.com/problems/classes-more-than-5-students/

"""
SELECT class
FROM Courses
GROUP BY class
HAVING count(student) >= 5;
"""

In [ ]:
# https://leetcode.com/problems/big-countries/submissions/1636368416/

"""
SELECT name, population, area
FROM World
WHERE area >= 3000000 OR population >= 25000000;
"""

In [ ]:
# https://leetcode.com/problems/customer-placing-the-largest-number-of-orders/

"""
SELECT customer_number
FROM Orders
GROUP BY customer_number
ORDER BY count(order_number) DESC
LIMIT 1;
"""

In [ ]:
# https://leetcode.com/problems/find-customer-referee/

"""
SELECT name
FROM Customer
WHERE referee_id != 2 OR referee_id IS NULL;
"""

In [ ]:
# https://leetcode.com/problems/employee-bonus/

"""
SELECT e.name, b.bonus
FROM Employee e 
LEFT JOIN Bonus b
ON e.empId = b.empId
WHERE b.bonus < 1000 OR b.bonus IS NULL;
"""

In [ ]:
# https://leetcode.com/problems/game-play-analysis-i/submissions/1629984989/

"""
SELECT player_id, min(event_date) as first_login
FROM Activity
GROUP BY player_id
ORDER BY player_id;
"""

In [ ]:
# https://leetcode.com/problems/rising-temperature/

"""
select w1.id 
from Weather w1
left join Weather w2
on w1.recordDate = w2.recordDate + INTERVAL '1 day'
where w1.temperature > w2.temperature;
"""

In [ ]:
# https://leetcode.com/problems/delete-duplicate-emails

"""
DELETE FROM Person p1
USING Person p2 
WHERE p1.email = p2.email
AND p1.id > p2.id;
"""

In [ ]:
# https://leetcode.com/problems/customers-who-never-order/

"""
SELECT c.name as Customers
FROM Customers c LEFT JOIN Orders o ON c.id = o.customerId
WHERE o.id IS NULL;
"""

In [ ]:
# https://leetcode.com/problems/second-highest-salary/

"""
select max(salary) as SecondHighestSalary
from Employee
where salary < (select max(salary) from Employee)
"""

In [ ]:
# https://leetcode.com/problems/employees-earning-more-than-their-manage

"""
SELECT e.name Employee
FROM Employee e
LEFT JOIN Employee m
ON e.managerId = m.id
WHERE e.managerId IS NOT NULL
AND e.salary > m.salary;
"""

In [ ]:
# https://leetcode.com/problems/duplicate-emails/

"""
postgresql []
SELECT email
FROM Person
GROUP BY email
HAVING count(*) > 1;
"""

In [ ]:
# https://leetcode.com/problems/combine-two-tables/

"""
SELECT p.firstName, p.lastName, a.city, a.state
FROM Person p
LEFT JOIN Address a
ON p.personId = a.personId
"""